In [1]:
# ! pip install wandb # colab only
import re
import os
from functools import reduce
import math
from collections import defaultdict
import numpy as np
import torch
from torch import nn
from torchinfo import summary
from pprint import pprint
import wandb
from sklearn.metrics import f1_score
import config as cfg

def download_and_unzip(url, save_dir='.'):
  # downloads and unzips url, if not already downloaded
  # used for downloading dataset and glove embeddings
  import os
  from urllib.request import urlopen
  from io import BytesIO
  from zipfile import ZipFile
  fname = url.split('/')[-1][:-4] if save_dir == '.' else save_dir
  if fname not in os.listdir():
    print(f'downloading and unzipping {fname}...', end=' ')
    r = urlopen(url)
    zipf = ZipFile(BytesIO(r.read()))
    zipf.extractall(path=save_dir)
    print(f'completed')
  else:
    print(f'{fname} already downloaded')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def get_wandbkey():
    with open('wandbkey.txt') as f:
        return f.read().strip()

In [2]:
PAD_TOKEN = 400000 # TODO used by collate_fn, can be done better probably
def get_glove(emb_size=100, number_token=False):
  """
    Download and load glove embeddings. 
    Parameters:
      emb_size: embedding size (50/100/200/300-dimensional vectors).    
    Returns tuple (voc, emb) where voc is dict from words to idx (in emb) and emb is (numpy) embedding matrix
  """
  n_tokens = 400000 + 1 # glove vocabulary size + PAD
  if emb_size not in (50, 100, 200, 300):
    raise ValueError(f'wrong size parameter: {emb_size}')
  
  if number_token: 
    n_tokens += 1
  download_and_unzip('http://nlp.stanford.edu/data/glove.6B.zip', save_dir='glove')
  vocabulary = dict()
  embedding_matrix = np.ones((n_tokens, emb_size))

  with open(f'glove/glove.6B.{emb_size}d.txt', encoding="utf8") as f:
    for i, line in enumerate(f):
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embedding_matrix[i] = coefs
        vocabulary[word] = i
  
  # add embedding for and padding and number token
  if number_token:
    embedding_matrix[n_tokens - 2] = 0
    vocabulary['<PAD>'] = n_tokens - 2
    digits = list(filter(lambda s: re.fullmatch('\d+(\.\d*)?', s) is not None, vocabulary.keys()))
    embedding_matrix[n_tokens - 1] = np.mean(embedding_matrix[[vocabulary[d] for d in digits]], axis=0)
    vocabulary['<NUM>'] = n_tokens - 1
  else: 
    embedding_matrix[n_tokens - 1] = 0
    vocabulary['<PAD>'] = n_tokens - 1
  return vocabulary, embedding_matrix

In [3]:
def load_classes():
    download_and_unzip('https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip')
    c = set()
    for doc in range(1, 200):
        with open(f'dependency_treebank/wsj_{doc:04d}.dp') as f:
            for line in f:
                if line.strip(): # check for empty lines
                    _, label, _ = line.split('\t')
                    #print(label)
                    c.add(label)
    return c

classes = {'$', 'NN', ',', 'RBS', 'FW', 'CC', '#', 'VBD', 'PRP', 'RBR', 'LS', ':', 'VBZ', 'MD',
           'EX', 'RB', 'WRB', 'NNS', 'VBG', 'PRP$', 'JJR', 'WP$', 'WP', '-LRB-', 'WDT', '``',
           '.', 'CD', 'JJ', "''", 'UH', 'VBN', 'IN', 'SYM', 'DT', 'JJS', '-RRB-', 'RP', 'VB',
           'POS', 'NNP', 'PDT', 'NNPS', 'VBP', 'TO', '<PAD>'}
punctuation_cls = {'$', ',', '#', ':', '-LRB-', '``', '.', "''", 'SYM', '-RRB-', '<PAD>'}
class2idx = {c: i for i, c in enumerate(classes)}
print(class2idx)

def add_oov(start_voc, oovs, embedding_matrix, sentences, verbose=True):
  """
    Computes new embedding matrix, adding embeddings for oovs
    Parameters:
      start_voc: dict, starting vocabulary that is extended with oovs
      oovs: set of string, oovs to add to the starting vocabulary and embedding matrix
      embedding_matrix: starting embedding matrix (numpy)
      sentences: list of list of strings, set used to compute oov embeddings
    Returns tuple (voc, emb) where voc is dict from words to idx (in emb) and emb is (numpy) embedding matrix with oovs
  """
  oovs = oovs - set(start_voc.keys())
  start_voc_size, emb_size = embedding_matrix.shape
  oov_embeddings = np.zeros((start_voc_size + len(oovs), emb_size))
  oov_embeddings[:start_voc_size] = embedding_matrix
  new_voc = dict(start_voc)

  for i, oov in enumerate(oovs):
    context_words = [new_voc[word] 
                    for sentence in filter(lambda s: oov in s, sentences)
                    for word in sentence if word in new_voc and word not in (oov, '<PAD>')]
    if verbose and len(context_words) == 0:
        print(f'Empty context for oov: {oov}')
        print([sentence for sentence in filter(lambda s: oov in s, sentences)])
    oov_embeddings[start_voc_size + i] = np.mean(oov_embeddings[context_words], axis=0)
    new_voc[oov] = start_voc_size + i
  return new_voc, oov_embeddings
    
def load_data(start, end, start_voc, embedding_matrix, number_token=False,
              drop_punctuation=False, split_docs=True, ret_counts=False):
  """
    Downloads dataset and preprocess data.
    Params:
      start: idx of first file to include in data
      end: idx of last file to include in data
      start_voc: starting vocabulary that is extended with oov terms
      embedding_matrix: embedding matrix that 
      number_token: if True, use a single token for all cardinal numbers
      drop_punctuation: if True, drop punt
      split_docs: if True, each sequence is one sentence; if false, each sequence is one document
      ret_counts: if True, also return counts of each word in the documents
    Returns 
  """
  # download dataset
  download_and_unzip('https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip')

  inputs, labels = [], []
  vocabulary = set()
  counts = defaultdict(int)
  
  # build dataset
  for doc in range(start, end+1):
    with open(f'dependency_treebank/wsj_{doc:04d}.dp') as f:
      
      input_seq, label_seq = [], []
      
      for line in f:
        if line.strip(): # check for empty lines
          word, label, _ = line.split('\t')
          word = word.lower()
          if '\/' in word:
            word = word.replace('\/', '-')
          if number_token and re.fullmatch('\d+(\.\d*)?', word) is not None:
            word = '<NUM>'
          if not drop_punctuation or label.isalpha(): # eventually drop punctuation
            vocabulary.add(word)
            input_seq.append(word)
            label_seq.append(label)
            counts[word] += 1
        elif split_docs: # sentence over, add to input if splitting documents
          inputs.append(input_seq)
          labels.append(label_seq)
          input_seq, label_seq = [], []

      inputs.append(input_seq)
      labels.append(label_seq)

  vocabulary, embedding_matrix = add_oov(start_voc, vocabulary, embedding_matrix, inputs)

  if ret_counts:
    return inputs, labels, vocabulary, embedding_matrix, counts
  else:
    return inputs, labels, vocabulary, embedding_matrix

{'JJS': 0, 'UH': 1, '<PAD>': 2, 'NNS': 3, 'NNPS': 4, 'RBS': 5, 'CC': 6, '``': 7, 'EX': 8, '-LRB-': 9, 'JJ': 10, 'POS': 11, 'NNP': 12, 'VBN': 13, "''": 14, '#': 15, 'VB': 16, 'RBR': 17, 'WDT': 18, 'VBD': 19, '$': 20, 'PRP': 21, 'CD': 22, 'LS': 23, 'RP': 24, 'NN': 25, 'PDT': 26, ':': 27, 'VBG': 28, 'VBP': 29, 'DT': 30, ',': 31, 'WRB': 32, 'WP': 33, 'WP$': 34, 'SYM': 35, 'MD': 36, 'JJR': 37, 'IN': 38, 'TO': 39, '-RRB-': 40, 'VBZ': 41, '.': 42, 'PRP$': 43, 'RB': 44, 'FW': 45}


In [4]:
class POSDataset(torch.utils.data.Dataset):
    """Simple dataset class to use dataloaders (batching) """
    def __init__(self, inputs, labels, vocabulary):
        self.inputs_str = inputs
        self.labels_str = labels
        self.voc = vocabulary
        self.inputs = [[vocabulary[word] for word in sequence] for sequence in inputs]
        self.labels = [[class2idx[label] for label in sequence] for sequence in labels]
    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]
    def __len__(self):
        return len(self.inputs)

def collate_fn(batch):
    """Used by DataLoader to pad batches"""
    seq_lens = torch.as_tensor([len(seq[0]) for seq in batch])
    padded_inputs = torch.nn.utils.rnn.pad_sequence([torch.as_tensor(seq[0]) for seq in batch], 
                                                     batch_first=True, padding_value=PAD_TOKEN)
    padded_targets = torch.nn.utils.rnn.pad_sequence([torch.as_tensor(seq[1]) for seq in batch], 
                                                     batch_first=True, padding_value=class2idx['<PAD>'])
    # inputs = []
    # targets = []
    # for _, t_seq in batch:
    #     if len(t_seq) > max_seq_len:
    #         targets.append(t_seq[:max_seq_len])
    #     else:
    #         targets.append(t_seq + [class2idx['<PAD>']] * (max_seq_len - len(t_seq)))
    return padded_inputs, padded_targets, seq_lens


class EarlyStopping:
    def __init__(self, patience, model, delta=0, path='res'):
        self.patience = patience
        self.delta = delta
        self.model = model
        self.path = path
        if not os.path.isdir(self.path):
            os.mkdir(self.path)
        self.best_score = float('inf')
        self.counter = 0
    
    def step(self, epoch, score):
        """ Updates ES tracker after one epoch.
            Params:
                epoch: current epoch
                score: validation loss
            Returns tuple (stop, checkpoint), 
                where stop is True if early stopping has occurred and False otherwise,
                and checkpoint is last best checkpoint
        """
        if score < self.best_score:
            # print('Validation loss decreasing, storing new checkpoint')
            self.best_score = score
            self.counter = 0
            checkpoint = {'model': self.model.state_dict(), 'epoch': epoch}
            torch.save(checkpoint, 'res/checkpoint.pth')
            return False, checkpoint
        elif abs(score - self.best_score) > self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                print(f'Early stopping occured at epoch {epoch} with patience {self.patience}')
                checkpoint = torch.load('res/checkpoint.pth', map_location='cpu')
                return True, checkpoint
            if self.counter == (self.patience//2):
                print(f'Validation loss increasing for {self.counter} epochs')
        return False, None

In [5]:
class POSTagger(torch.nn.Module):

  def __init__(self, embedding_matrix, type, rec_size=1, units=None, hid_size=50):
    """
      A recurrent network performing multiclass classification (POS tagging).
      Params:
        type: type of rnn, either 'lstm' or 'gru'
        embedding_matrix: embedding matrix for embedding layer
        rec_size: number of stacked recurrent modules
        units: int or None, if given then add one additional linear layer with given number of units
        hid_size: size of hidden state of recurrent module
    """
    super().__init__()

    emb_size = embedding_matrix.shape[1]
    self.emb_layer = nn.Embedding.from_pretrained(torch.as_tensor(embedding_matrix))

    if type == 'lstm':
      rec_module = nn.LSTM
    elif type == 'gru':
      rec_module = nn.GRU
    else:
      raise ValueError(f'wrong type {type}, either lstm or gru')
    self.rec_modules = rec_module(input_size=emb_size, hidden_size=hid_size, bidirectional=True, batch_first=True, num_layers=rec_size)
    self.fc_modules = nn.Sequential(nn.Linear(2 * hid_size, units if units is not None else len(classes)))
    if units is not None:
      self.fc_modules.add_module('fc1_relu', nn.ReLU())
      self.fc_modules.add_module('fc2', nn.Linear(units, len(classes)))


  def forward(self, x, seq_lens):
    embed_vecs = self.emb_layer(x).float()
    packed_vecs = torch.nn.utils.rnn.pack_padded_sequence(embed_vecs, seq_lens, batch_first=True, enforce_sorted=False)
    rec_out, _ = self.rec_modules(packed_vecs)
    unpacked_rec_out, _ = torch.nn.utils.rnn.pad_packed_sequence(rec_out, batch_first=True, padding_value=PAD_TOKEN)
    fc_out = self.fc_modules(unpacked_rec_out)
    return fc_out

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, ignore_index=-100, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduce = reduce
        self.ignore_index = ignore_index
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=self.ignore_index, reduction='none')

    def forward(self, inputs, targets): 
        ce_loss = self.loss_fn(inputs, targets)

        pt = torch.exp(-ce_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * ce_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [6]:
def train_one_epoch(model, optimizer, loss_fn, data_loader, device):
    """ 
        Trains model for one epoch on the given dataloader.
        Parameters:
            model: torch.nn.Module to train
            optimizer: torch.optim optimizer object
            loss_fn: torch.nn criterion to use to compute loss, given outputs and targets
            data_loader: torch.utils.data.DataLoader 
            device: torch.device where training is performed
        Returns log dict {'train/loss' : list(loss values for each batch)} 
    """
    model.train()
    log_dict = {'train/loss': []}

    for inputs, targets, seq_lens in data_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        scores = model(inputs, seq_lens).transpose(1, 2)
        loss = loss_fn(scores, targets)
        loss_value = loss.item()

        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            exit(1)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        log_dict['train/loss'].append(loss_value)

    return log_dict

def evaluate(model, loss_fn, data_loader, device, split, ret_f1_classes=False):
    """ 
        Evaluate model on the given dataloader.
        Parameters:
            model: torch.nn.Module to evaluate
            loss_fn: torch.nn criterion to use to compute loss, given outputs and targets
            data_loader: torch.utils.data.DataLoader 
            device: torch.device where evaluation is performed
            split: either 'valid' or 'test'
            ret_f1_classes: if True, also returns per-class f1 scores
        Returns log dict {'valid/loss' : mean loss, 'valid/{metric}': mean metric} 
    """
    model.eval()
    assert len(data_loader) == 1 # must be a single batch
    with torch.no_grad():
        inputs, targets, seq_lens = next(iter(data_loader))
        inputs = inputs.to(device)
        targets = targets.to(device)

        scores = model(inputs, seq_lens).transpose(1, 2)
        losses = loss_fn(scores, targets).item()
        preds = torch.argmax(scores, 1)

        targets = targets.cpu().numpy()
        preds = preds.cpu().numpy()
        mask = [targets != class2idx[c] for c in punctuation_cls]
        mask = np.array(reduce(lambda a,b: a & b, mask)).reshape(targets.shape)
        acc = np.where(mask, targets==preds, False).sum() / mask.sum()
        cls = [class2idx[c] for c in (classes - punctuation_cls)]
        f1_classes = f1_score(targets.reshape(-1), preds.reshape(-1),
                      labels=cls, average=None, zero_division=1)

    log_dict = {f'{split}/loss': losses,
                f'{split}/accuracy': acc,
                f'{split}/f1': np.mean(f1_classes)}
    if ret_f1_classes:
        return log_dict, {c:s for c,s in zip(cls, f1_classes)}
    else:
        return log_dict

In [7]:
def train(tags=None, verbose=False, test=False, number_token=False, focal_loss=False):
    """ Fully trains one model, based on cfg parameters, on training set and performs evaluation on validation set.
        Returns trained model.
    """
    idx2classes = {i: c for c, i in class2idx.items()}
    cfg_dict = {
        'epochs': cfg.EPOCHS, 'batch_size': cfg.BATCH_SIZE, 'number_token': number_token,
        'model': cfg.TYPE, 'rec_size': cfg.REC_SIZE, 'units': cfg.UNITS, 'hid_size': cfg.HID_SIZE,
        'optim': cfg.OPTIM, 'lr': cfg.LR, 'alpha': cfg.ALPHA, 'betas': cfg.BETAS, 'momentum': cfg.MOMENTUM, 'focal_loss': focal_loss
    }
    if verbose:
        print('CONFIG PARAMETERS:')
        pprint(cfg_dict)
    wandb.login(key=get_wandbkey())
    run = wandb.init(project="assignment-one", entity="nlpetroni", group=f'{"testing" if test else "validation"}',
                     reinit=True, config=cfg_dict, tags=tags)
    wandb.define_metric("train_step")
    wandb.define_metric("epoch")
    wandb.define_metric('train/loss', step_metric="train_step", summary="min")
    wandb.define_metric(f"valid/loss", step_metric="epoch", summary="min")
    wandb.define_metric(f"valid/accuracy", step_metric="epoch", summary="max")
    wandb.define_metric(f"valid/f1", step_metric="epoch", summary="max")
    wandb.define_metric(f"test/accuracy", step_metric="epoch", summary="max")
    wandb.define_metric(f"test/f1", step_metric="epoch", summary="max")

    glove_voc, embedding_matrix = get_glove(number_token=number_token)
    if not test:
        split = 'valid'
        train_set, train_labels, train_voc, embedding_matrix = load_data(1, 100, glove_voc, embedding_matrix, number_token=number_token, drop_punctuation=False)
        valid_set, valid_labels, valid_voc, embedding_matrix = load_data(101, 150, train_voc, embedding_matrix, number_token=number_token, drop_punctuation=False)
        train_ds = POSDataset(train_set, train_labels, train_voc)
        valid_ds = POSDataset(valid_set, valid_labels, valid_voc)
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=cfg.BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=len(valid_ds), collate_fn=collate_fn)
    else:
        split = 'test'
        train_set, train_labels, train_voc, embedding_matrix = load_data(1, 150, glove_voc, embedding_matrix, number_token=number_token, drop_punctuation=False)
        test_set, test_labels, test_voc, embedding_matrix = load_data(151, 199, train_voc, embedding_matrix, number_token=number_token, drop_punctuation=False)
        train_ds = POSDataset(train_set, train_labels, train_voc)
        test_ds = POSDataset(test_set, test_labels, test_voc)
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=cfg.BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=len(test_ds), collate_fn=collate_fn)


    model = POSTagger(embedding_matrix, type=cfg.TYPE, rec_size=cfg.REC_SIZE, units=cfg.UNITS, hid_size=cfg.HID_SIZE).to(device)
    wandb.watch(model, log_graph=True)
    if verbose:
        print(summary(model))

    params = [p for p in model.parameters() if p.requires_grad]
    if cfg.OPTIM == 'rmsprop':
        optimizer = torch.optim.RMSprop(params, lr=cfg.LR, alpha=cfg.ALPHA, momentum=cfg.MOMENTUM, weight_decay=cfg.WEIGHT_DECAY)
    elif cfg.OPTIM == 'adam':
        optimizer = torch.optim.Adam(params, lr=cfg.LR, betas=cfg.BETAS, weight_decay=cfg.WEIGHT_DECAY)
    else:
        raise ValueError(f'wrong optim {cfg.OPTIM}, either rmsprop or adam')
    if focal_loss: 
        loss_fn = FocalLoss(ignore_index=class2idx['<PAD>'])
    else:
        loss_fn = nn.CrossEntropyLoss(ignore_index=class2idx['<PAD>'])
    train_step = 0
    es_tracker = EarlyStopping(10, model)
    epoch = 0
    stop = False
    print('STARTING TRAINING')
    while epoch < cfg.EPOCHS and not stop:
        train_log_dict = train_one_epoch(model, optimizer, loss_fn, train_dl, device)
        if not test:
            valid_log_dict, f1_classes = evaluate(model, loss_fn, valid_dl, device, split=split, ret_f1_classes=True)
            stop, checkpoint = es_tracker.step(epoch, valid_log_dict['valid/loss'])
            wandb.log({'epoch': epoch, 'valid/loss': valid_log_dict['valid/loss'],
                       'valid/accuracy': valid_log_dict['valid/accuracy'], 'valid/f1': valid_log_dict['valid/f1'],
                       'valid/f1_distribution': wandb.Histogram(np_histogram=np.histogram(list(f1_classes.values())))})
            if stop:
                model.load_state_dict(checkpoint['model'])
            if (epoch % 25) == 0:
                print(f'[{epoch:03d}/{cfg.EPOCHS:03d}] train loss: {np.mean(train_log_dict["train/loss"]):.3f}, valid loss: {valid_log_dict["valid/loss"]:.3f}, f1: {valid_log_dict["valid/f1"]:.2f} accuracy: {valid_log_dict["valid/accuracy"]:.2f}')
        for batch_loss in train_log_dict['train/loss']:
            wandb.log({'train_step': train_step, 'epoch': epoch, 'train/loss': batch_loss})
            train_step += 1
        epoch += 1
    # log per-class f1 scores
    data = [[idx2classes[i], score] for i, score in f1_classes.items()]
    table = wandb.Table(data=data, columns=["class", "f1_score"])
    wandb.log({'valid/f1_per_class': wandb.plot.bar(table, "class", "f1_score", title="F1 per class bar chart")})
    if test:
        log_dict, f1_classes = evaluate(model, loss_fn, test_dl, device, split=split, ret_f1_classes=True)
        data = [[idx2classes[i], score] for i,score in f1_classes.items()]
        table = wandb.Table(data=data, columns = ["class", "f1_score"])
        wandb.log({'test/loss': log_dict['test/loss'], 'test/accuracy': log_dict['test/accuracy'], 'test/f1': log_dict['test/f1'],
                   'test/f1_per_class': wandb.plot.bar(table, "class", "f1_score", title="F1 per class bar chart")})

    run.finish()

    return model, run

In [10]:
# hyperparameter tuning
for (tag, type, rec_size, units) in (('lstm_1L', 'lstm', 1, None), ('lstm_2L', 'lstm', 2, None),
                                     ('fc_2L', 'lstm', 1, 128), ('gru', 'gru', 1, None)):
    for optim in ('rmsprop', 'adam'):
        for fl in (True, False):
            for hid_size in (32, 64):
                cfg.TYPE = type
                cfg.REC_SIZE = rec_size
                cfg.UNITS = units
                cfg.OPTIM = optim
                cfg.HID_SIZE = hid_size
                _, run = train(tags=[tag], focal_loss=fl)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/loss,█▆▆▆▅▅▃▄▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁
train_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
valid/accuracy,▁▃▄▅▆▆▇▇▇███████
valid/f1,▁▂▃▄▅▅▆▆▇▇▇▇████
valid/loss,█▆▅▄▃▂▂▂▂▁▁▁▁▁▁▁
epoch,15
train_step,991


wandb: wandb version 0.12.8 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


glove already downloaded
dependency_treebank already downloaded
dependency_treebank already downloaded


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/diego/.netrc
wandb: wandb version 0.12.8 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


glove already downloaded
dependency_treebank already downloaded
dependency_treebank already downloaded


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


STARTING TRAINING
[000/200] train loss: 3.484, valid loss: 3.095, f1: 0.04 accuracy: 0.27
[025/200] train loss: 0.429, valid loss: 0.473, f1: 0.59 accuracy: 0.86
[050/200] train loss: 0.263, valid loss: 0.336, f1: 0.67 accuracy: 0.89
[075/200] train loss: 0.183, valid loss: 0.290, f1: 0.75 accuracy: 0.91
[100/200] train loss: 0.129, valid loss: 0.275, f1: 0.75 accuracy: 0.91
Validation loss increasing for 5 epochs
Early stopping occured at epoch 116 with patience 10


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/loss,█▆▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
valid/accuracy,▁▃▅▆▇▇▇▇▇▇██████████████████████████████
valid/f1,▁▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████████████████
valid/loss,█▆▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,116
train_step,7253


In [ ]:
# test best model
# first best model
cfg.REC_SIZE = 1
cfg.UNITS = 128
train(test=True)
# second best model
cfg.REC_SIZE = 2
cfg.UNITS = None
train(test=True)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,███▇▇▇▇▆▆▅▅▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▁▁▁
train_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,0
train_step,101


wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


glove already downloaded
dependency_treebank already downloaded
dependency_treebank already downloaded


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


...

glove already downloaded
dependency_treebank already downloaded
dependency_treebank already downloaded


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


STARTING TRAINING


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test/accuracy,▁
test/f1,▁
test/loss,▁
train/loss,█▃▃▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,99
test/loss,0.32036
train_step,10199


(POSTagger(
   (emb_layer): Embedding(400661, 100)
   (rec_modules): LSTM(100, 64, num_layers=2, batch_first=True, bidirectional=True)
   (fc_modules): Sequential(
     (fc_0): Linear(in_features=128, out_features=46, bias=True)
   )
 ),
 <wandb.sdk.wandb_run.Run at 0x7f7ff4604a00>)